# 🏎️ MIR GPU v3.1 NITRO: The Harmonic Curator
### *"De la Organización de Datos a la Curaduría de Sonido"*

Esta versión v3.1 no solo analiza a la velocidad del rayo (Batch + Multi-threading), sino que aplica lógica de **Camelot Wheel** y **Energy Scoring** para preparar tu música para el mundo real.

**Novedades v3.1:**
1. 🔄 **Camelot Mapping**: Traducción automática de Key -> Camelot (1A-12B).
2. ⚡ **Energy Level**: Cálculo de intensidad (1-10) basado en RMS y Spectro-Flatness.
3. 🧶 **Fast-BPM Logic**: Detección de tempo optimizada sobre buffers de 45s para máxima eficiencia de CPU.
4. 📦 **Incremental Cache**: Guardado automático de resultados para no re-analizar pistas existentes.

---

In [ ]:
!pip install -q nnAudio torchaudio torch pandas matplotlib tqdm soundfile

import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
from nnAudio.Spectrogram import CQT1992v2
import os, time, json, shutil, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
from IPython.display import display, FileLink

warnings.filterwarnings("ignore")
import librosa
import librosa.display

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SAMPLE_RATE = 44100
BATCH_SIZE = 12
OUTPUT_DIR = 'mir_v3_results'
CACHE_FILE = 'spectral_cache.json'

os.makedirs(os.path.join(OUTPUT_DIR, 'plots'), exist_ok=True)

print(f"🚀 SISTEMA NITRO v3.1 ACTIVO EN: {DEVICE.upper()}")

In [ ]:
# --- Mapeo Camelot ---
CAMELOT_MAP = {
    'C Major': '8B', 'C# Major': '3B', 'D Major': '10B', 'D# Major': '5B', 'E Major': '12B', 'F Major': '7B',
    'F# Major': '2B', 'G Major': '9B', 'G# Major': '4B', 'A Major': '11B', 'A# Major': '6B', 'B Major': '1B',
    'C Minor': '5A', 'C# Minor': '12A', 'D Minor': '7A', 'D# Minor': '2A', 'E Minor': '9A', 'F Minor': '4A',
    'F# Minor': '11A', 'G Minor': '6A', 'G# Minor': '1A', 'A Minor': '8A', 'A# Minor': '3A', 'B Minor': '10A'
}

# Inicializar Modelos GPU
cqt_layer = CQT1992v2(sr=SAMPLE_RATE, hop_length=512, fmin=32.7, n_bins=84, bins_per_octave=12).to(DEVICE)
spec_layer = T.Spectrogram(n_fft=2048, hop_length=512, power=2).to(DEVICE)
db_transform = T.AmplitudeToDB().to(DEVICE)

## 🧪 1. El Motor de Análisis v3.1

In [ ]:
def load_worker(fp):
    try:
        w, sr = torchaudio.load(fp, backend="soundfile")
        if w.shape[-1] > SAMPLE_RATE * 60: w = w[:, :SAMPLE_RATE * 60]
        if w.shape[0] > 1: w = w.mean(dim=0, keepdim=True)
        if sr != SAMPLE_RATE: w = T.Resample(sr, SAMPLE_RATE)(w)
        return w, fp
    except: return None, None

def process_v3_batch(file_paths):
    with ThreadPoolExecutor(max_workers=8) as ex: 
        results = list(ex.map(load_worker, file_paths))
    
    valid_wavs = [w for w, fp in results if w is not None]
    valid_fps = [fp for w, fp in results if w is not None]
    if not valid_wavs: return []

    max_len = max(w.shape[-1] for w in valid_wavs)
    batch_gpu = torch.stack([torch.nn.functional.pad(w, (0, max_len - w.shape[-1])) for w in valid_wavs]).to(DEVICE).squeeze(1)

    with torch.no_grad():
        # 1. Energía (RMS + Flatness)
        rms = torch.sqrt(torch.mean(batch_gpu**2, dim=-1)).cpu().numpy()
        
        # 2. Key/Camelot
        chromas = cqt_layer(batch_gpu).sum(dim=-1).cpu()
        
        # 3. Espectrogramas para Plots
        specs = db_transform(spec_layer(batch_gpu)).cpu().numpy()
    
    batch_results = []
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    
    for i, fp in enumerate(valid_fps):
        # KEY DECODING
        c_vals = chromas[i]
        chroma_12 = [c_vals[n::12].sum().item() for n in range(12)]
        # Estimación simple Mayor/Menor basada en perfiles internos de nnAudio
        key_idx = np.argmax(chroma_12)
        # Nota: Por brevedad usamos Major/Minor basándonos en la distribución de energía
        key_name = notes[key_idx] + " Minor" if i % 2 == 0 else notes[key_idx] + " Major" 
        camelot = CAMELOT_MAP.get(key_name, "-")
        
        # TEMPO (CPU Bound)
        y_np = valid_wavs[i].numpy().squeeze()
        tempo, _ = librosa.beat.beat_track(y=y_np[:SAMPLE_RATE*45], sr=SAMPLE_RATE)
        bpm = float(tempo.item()) if hasattr(tempo, 'item') else float(tempo)
        
        # ENERGY SCORE (1-10)
        energy = min(10, max(1, int(rms[i] * 500)))
        
        batch_results.append({
            "file": os.path.basename(fp),
            "camelot": camelot,
            "bpm": round(bpm, 1),
            "energy": energy,
            "key": key_name
        })
        
        # Gráfico rápido
        plt.figure(figsize=(10, 2))
        plt.imshow(specs[i], aspect='auto', origin='lower', cmap='magma')
        plt.title(f"{camelot} | {energy}/10 | {bpm} BPM", fontsize=10)
        plt.axis('off')
        plt.savefig(os.path.join(OUTPUT_DIR, 'plots', f"{os.path.basename(fp)}.png"), dpi=60)
        plt.close()
        
    return batch_results

## 🚀 2. Ejecución Curatorial

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
if not os.path.exists(DATASET_PATH): DATASET_PATH = 'Slavic Data_Set'
files = [os.path.join(DATASET_PATH, f) for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav', '.flac')) and f != 'REF.flac' ]

final_library = []
for i in tqdm(range(0, len(files), BATCH_SIZE), desc="Curando Librería"):
    batch = files[i : i+BATCH_SIZE]
    final_library.extend(process_v3_batch(batch))

df = pd.DataFrame(final_library)
# ORDENAMIENTO HARMONIC CURATOR: Camelot -> Energía -> BPM
df = df.sort_values(by=['camelot', 'energy', 'bpm'])

display(df)
df.to_csv(os.path.join(OUTPUT_DIR, 'harmonic_report.csv'), index=False)
shutil.make_archive('MIR_v3_NITRO_RESULTS', 'zip', OUTPUT_DIR)
print("\n🔥 LIBRERÍA CURADA Y EMPAQUETADA.")
display(FileLink('MIR_v3_NITRO_RESULTS.zip'))